In [1]:
#load libraries
import os
import numpy as np
import pickle
import argparse
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve
from sklearn.mixture import GaussianMixture as GMM
import pandas as pd


In [2]:
#Load training data 
#'/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/train_sample.pkl'

In [3]:
def extract_pkl(path,feature_type):
    X = []
    y = []

    max_len = 50  # 1.25 seconds  # check the timesteps of cqcc and mfcc 
    lens = []
    with open(path, 'rb') as infile:
        data = pickle.load(infile)
        for feat_cqcc, feat_mfcc, label in data:
            #mfcc = mfccs.mean(0) # sum over all timesteps for now    # timesteps X 13
            #lens.append(mfccs.shape[0])
            if feature_type == "cqcc":
                feats = feat_cqcc
            elif feature_type == "mfcc":
                feats = feat_mfcc
            num_dim = feats.shape[1]
            if len(feats) > max_len:
                feats = feats[:max_len]
            elif len(feats) < max_len:
                feats = np.concatenate((feats, np.array([[0.]*num_dim]*(max_len-len(feats)))), axis=0)
            X.append(feats.reshape(-1))
            y.append(label)
    X = np.array(X)
    return X,y

In [4]:
X_train_mfcc,y_train_mfcc = extract_pkl('/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/train_sample.pkl','mfcc')

In [5]:
X_train_mfcc.shape

(6, 650)

In [7]:
X_train_mfcc[0].shape

(650,)

In [14]:
X_train_cqcc,y_train_cqcc = extract_pkl('/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/train_sample.pkl','cqcc')

In [15]:
X_train_cqcc.shape

(6, 3000)

In [ ]:
#load test data

In [6]:
X_test_mfcc,y_test_mfcc = extract_pkl('/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/dev_sample.pkl','mfcc')

In [7]:
X_test_cqcc,y_test_cqcc = extract_pkl('/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/dev_sample.pkl','cqcc')

In [ ]:
# load evaluation data

In [4]:
gmm_mfcc = GMM(n_components = 144, covariance_type='diag',n_init = 50)
gmm_mfcc.fit(X_train_mfcc, y_train_mfcc)

NameError: name 'X_train_mfcc' is not defined

In [ ]:
#MFCC features of each dataset

In [ ]:
# CQCC features of each dataset

In [ ]:
#Classifiers training

In [36]:
clf_mfcc = SVC(probability=True)
clf_mfcc.fit(X_train_mfcc, y_train_mfcc)

SVC(probability=True)

In [19]:
clf_cqcc = SVC()
clf_cqcc.fit(X_train_cqcc, y_train_cqcc)

SVC()

In [ ]:
#Classifer testing

In [37]:
clf_mfcc.score(X_test_mfcc, y_test_mfcc)

0.7142857142857143

In [21]:
clf_cqcc.score(X_test_cqcc, y_test_cqcc)

0.8571428571428571

In [41]:
#EER

In [51]:
def eer(model,X,y):
    y_score = model.decision_function(X)
    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label='spoof')
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    print ('EER', eer)


In [52]:
eer(clf_mfcc,X_test_mfcc, y_test_mfcc)

EER 0.25


In [53]:
eer(clf_cqcc,X_test_cqcc, y_test_cqcc)

EER 0.0


In [ ]:
# (t-DCF) calculation

In [ ]:
import sys
import numpy as np
import eval_metrics as em
import matplotlib.pyplot as plt

In [ ]:
# Replace CM scores with your own scores or provide score file as the first argument.
cm_score_file = 'scores/cm_dev.txt'
# Replace ASV scores with organizers' scores or provide score file as the second argument.
asv_score_file = 'scores/asv_dev.txt'

In [ ]:
# Fix tandem detection cost function (t-DCF) parameters
Pspoof = 0.05
cost_model = {
    'Pspoof': Pspoof,  # Prior probability of a spoofing attack
    'Ptar': (1 - Pspoof) * 0.99,  # Prior probability of target speaker
    'Pnon': (1 - Pspoof) * 0.01,  # Prior probability of nontarget speaker
    'Cmiss_asv': 1,  # Cost of ASV system falsely rejecting target speaker
    'Cfa_asv': 10,  # Cost of ASV system falsely accepting nontarget speaker
    'Cmiss_cm': 1,  # Cost of CM system falsely rejecting target speaker
    'Cfa_cm': 10,  # Cost of CM system falsely accepting spoof
}

# Load organizers' ASV scores
asv_data = np.genfromtxt(asv_score_file, dtype=str)
asv_sources = asv_data[:, 0]
asv_keys = asv_data[:, 1]
asv_scores = asv_data[:, 2].astype(np.float)

# Load CM scores
cm_data = np.genfromtxt(cm_score_file, dtype=str)
cm_utt_id = cm_data[:, 0]
cm_sources = cm_data[:, 1]
cm_keys = cm_data[:, 2]
cm_scores = cm_data[:, 3].astype(np.float)

# Extract target, nontarget, and spoof scores from the ASV scores
tar_asv = asv_scores[asv_keys == 'target']
non_asv = asv_scores[asv_keys == 'nontarget']
spoof_asv = asv_scores[asv_keys == 'spoof']

# Extract bona fide (real human) and spoof scores from the CM scores
bona_cm = cm_scores[cm_keys == 'bonafide']
spoof_cm = cm_scores[cm_keys == 'spoof']

# EERs of the standalone systems and fix ASV operating point to EER threshold
eer_asv, asv_threshold = em.compute_eer(tar_asv, non_asv)
eer_cm = em.compute_eer(bona_cm, spoof_cm)[0]


[Pfa_asv, Pmiss_asv, Pmiss_spoof_asv] = em.obtain_asv_error_rates(tar_asv, non_asv, spoof_asv, asv_threshold)


# Compute t-DCF
tDCF_curve, CM_thresholds = em.compute_tDCF(bona_cm, spoof_cm, Pfa_asv, Pmiss_asv, Pmiss_spoof_asv, cost_model, True)

# Minimum t-DCF
min_tDCF_index = np.argmin(tDCF_curve)
min_tDCF = tDCF_curve[min_tDCF_index]


print('ASV SYSTEM')
print('   EER            = {:8.5f} % (Equal error rate (target vs. nontarget discrimination)'.format(eer_asv * 100))
print('   Pfa            = {:8.5f} % (False acceptance rate of nontargets)'.format(Pfa_asv * 100))
print('   Pmiss          = {:8.5f} % (False rejection rate of targets)'.format(Pmiss_asv * 100))
print('   1-Pmiss,spoof  = {:8.5f} % (Spoof false acceptance rate)'.format((1 - Pmiss_spoof_asv) * 100))

print('\nCM SYSTEM')
print('   EER            = {:8.5f} % (Equal error rate for countermeasure)'.format(eer_cm * 100))

print('\nTANDEM')
print('   min-tDCF       = {:8.5f}'.format(min_tDCF))


# Visualize ASV scores and CM scores
plt.figure()
ax = plt.subplot(121)
plt.hist(tar_asv, histtype='step', density=True, bins=50, label='Target')
plt.hist(non_asv, histtype='step', density=True, bins=50, label='Nontarget')
plt.hist(spoof_asv, histtype='step', density=True, bins=50, label='Spoof')
plt.plot(asv_threshold, 0, 'o', markersize=10, mfc='none', mew=2, clip_on=False, label='EER threshold')
plt.legend()
plt.xlabel('ASV score')
plt.ylabel('Density')
plt.title('ASV score histogram')

ax = plt.subplot(122)
plt.hist(bona_cm, histtype='step', density=True, bins=50, label='Bona fide')
plt.hist(spoof_cm, histtype='step', density=True, bins=50, label='Spoof')
plt.legend()
plt.xlabel('CM score')
#plt.ylabel('Density')
plt.title('CM score histogram')


# Plot t-DCF as function of the CM threshold.
plt.figure()
plt.plot(CM_thresholds, tDCF_curve)
plt.plot(CM_thresholds[min_tDCF_index], min_tDCF, 'o', markersize=10, mfc='none', mew=2)
plt.xlabel('CM threshold index (operating point)')
plt.ylabel('Norm t-DCF');
plt.title('Normalized tandem t-DCF')
plt.plot([np.min(CM_thresholds), np.max(CM_thresholds)], [1, 1], '--', color='black')
plt.legend(('t-DCF', 'min t-DCF ({:.5f})'.format(min_tDCF), 'Arbitrarily bad CM (Norm t-DCF=1)'))
plt.xlim([np.min(CM_thresholds), np.max(CM_thresholds)])
plt.ylim([0, 1.5])

plt.show()

In [ ]:
# Ensemble Cqcc & Mfcc

In [ ]:
cqcc vs mfcc

In [6]:
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import soundfile as sf                                                      


In [41]:
#(rate,sig) = wav.read()
path = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/ASVspoof2019_LA_eval/flac/LA_E_A8192912.flac'
data, samplerate = sf.read(path) 
mfcc_feat = mfcc(data,samplerate)

In [40]:
pd.DataFrame(mfcc_feat)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-13.274142,-9.781909,11.773378,6.488668,14.412670,12.228407,14.443465,5.295808,2.497549,-5.828939,-3.704579,2.192505,-0.489304
1,-13.386678,-8.632389,12.743090,7.447218,12.156694,11.552882,13.041253,11.414261,6.602971,-2.677074,-2.258512,-2.558524,-5.621858
2,-13.308330,-11.130632,12.713072,3.550921,6.667538,17.143103,18.444377,11.802344,14.166637,5.169349,0.140413,-3.394843,-5.348122
3,-13.170443,-10.242958,18.024891,8.308462,7.884839,16.331613,15.491242,7.621745,2.097887,-2.694678,-2.183576,-0.701587,0.727929
4,-13.036753,-7.855870,18.164287,7.286864,10.357723,13.243471,8.654623,8.281565,8.822903,-3.094381,-7.518918,2.152017,-2.413962
...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,-13.238128,-7.352174,15.502873,0.284284,0.235365,2.239008,3.589126,0.160059,-0.724364,-3.870127,5.175457,-4.814993,-9.201507
413,-13.319352,-8.878909,15.538589,1.611144,-2.287456,-2.881913,-2.102067,-1.247424,4.370124,0.128309,-2.008788,-0.658629,-4.578762
414,-13.341226,-8.049504,12.699502,2.759916,1.108374,-1.696594,-3.728400,2.255740,3.115443,3.984150,0.540114,-2.969240,-3.159250
415,-13.512480,-12.592836,8.218552,2.798527,5.540127,-1.323482,-1.876995,4.453344,5.735194,6.918918,5.262617,1.045396,2.732618


In [43]:
pd.DataFrame(X_train_mfcc)

,0,1,2,3,4,5,6,7,8,9,...,640,641,642,643,644,645,646,647,648,649
0,-12.079208,-10.031430,9.425462,2.853750,6.814613,5.432624,2.968917,5.024234,5.435427,-0.403769,...,0.889986,-3.084318,-1.719646,5.256726,4.218211,-1.406790,2.278960,2.529413,1.126907,-1.235019
1,-13.242612,-10.545304,5.043834,0.150626,3.147927,2.871703,7.430171,-0.128426,2.008671,8.247766,...,2.564975,2.011975,6.261104,11.665453,5.426103,4.988994,3.184435,-1.244950,-3.156187,0.217522
2,-12.702974,-10.056060,11.439114,3.113770,2.907922,3.778411,8.533606,11.772646,9.305914,5.113260,...,1.854354,-0.158401,0.938198,6.466008,0.372584,2.895222,-3.439668,-4.094876,-2.883989,-2.521317
3,-14.214596,-31.567461,-4.517729,-3.030669,4.520188,1.991540,9.000989,12.429638,14.707430,16.775602,...,7.954628,17.206805,13.202963,14.275127,16.214005,20.235629,20.639901,18.535655,12.019987,8.923783
4,-14.986509,-27.206089,-1.328281,-2.767894,4.493338,4.608320,10.184276,9.410030,9.998180,8.772871,...,11.827195,18.421496,9.661060,8.155932,10.599895,6.507028,9.503384,7.044436,6.968312,11.159347
5,-15.427970,-28.426288,-0.632804,-1.002060,6.284443,5.711783,7.843549,4.020173,2.956051,8.819512,...,15.106872,19.000094,8.902957,9.757914,11.416898,6.730601,4.083023,2.373649,-1.145819,7.434624
